In [22]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import datetime
import pandas as pd

In [29]:
data = {"dates" : [],
        "values" : []
        }

In [30]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )          
             
        counter+=1

        if counter == 5:
            counter = 0
            iter += 1
            avg_y = (landmarks[15].y+landmarks[16].y+landmarks[17].y+landmarks[18].y+
                        landmarks[19].y+landmarks[20].y+landmarks[21].y+landmarks[22].y)/9

            data["values"].append(avg_y)
            data["dates"].append(datetime.datetime.today().strftime("%d-%m-%y %H:%M:%S"))
            if iter == 180:
                break
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [31]:
data

{'dates': ['11-02-23 11:51:35',
  '11-02-23 11:51:36',
  '11-02-23 11:51:36',
  '11-02-23 11:51:36',
  '11-02-23 11:51:36',
  '11-02-23 11:51:37',
  '11-02-23 11:51:37',
  '11-02-23 11:51:37',
  '11-02-23 11:51:37',
  '11-02-23 11:51:38',
  '11-02-23 11:51:38',
  '11-02-23 11:51:38',
  '11-02-23 11:51:38',
  '11-02-23 11:51:39',
  '11-02-23 11:51:39',
  '11-02-23 11:51:39',
  '11-02-23 11:51:39',
  '11-02-23 11:51:40',
  '11-02-23 11:51:40',
  '11-02-23 11:51:40',
  '11-02-23 11:51:40',
  '11-02-23 11:51:41',
  '11-02-23 11:51:41',
  '11-02-23 11:51:41',
  '11-02-23 11:51:41',
  '11-02-23 11:51:42',
  '11-02-23 11:51:42',
  '11-02-23 11:51:42',
  '11-02-23 11:51:42',
  '11-02-23 11:51:43',
  '11-02-23 11:51:43',
  '11-02-23 11:51:43',
  '11-02-23 11:51:43',
  '11-02-23 11:51:44',
  '11-02-23 11:51:44',
  '11-02-23 11:51:44',
  '11-02-23 11:51:44',
  '11-02-23 11:51:45',
  '11-02-23 11:51:45',
  '11-02-23 11:51:45',
  '11-02-23 11:51:45',
  '11-02-23 11:51:46',
  '11-02-23 11:51:46',
  

In [32]:
df = pd.DataFrame(data = data)

In [33]:
df

,dates,values
0,11-02-23 11:51:35,0.448010
1,11-02-23 11:51:36,0.386415
2,11-02-23 11:51:36,0.369966
3,11-02-23 11:51:36,0.374102
4,11-02-23 11:51:36,0.405868
...,...,...
175,11-02-23 11:52:19,0.371755
176,11-02-23 11:52:19,0.370815
177,11-02-23 11:52:19,0.376193
178,11-02-23 11:52:19,0.385646


In [34]:
df.to_csv("y_axis_2.csv" , index = False)